In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# DATA

In [2]:
# Expected return
mu = np.array([0.01, -0.01, 0.03, -0.03, 0.05, -0.05])
# Covariance matrix
sigma_vals = np.array([
    0.06101, 0.02252, 0.03315, 0.03971, 0.04186, 0.04520,
    0.02252, 0.08762, 0.04137, 0.04728, 0.05241, 0.05310,
    0.03315, 0.04137, 0.10562, 0.06210, 0.06885, 0.06574,
    0.03971, 0.04728, 0.06210, 0.11357, 0.07801, 0.07790,
    0.04186, 0.05241, 0.06885, 0.07801, 0.19892, 0.09424,
    0.04520, 0.05310, 0.06574, 0.07790, 0.09424, 0.36240
])
sigma = sigma_vals.reshape((6, 6))  # 6x6 covariance matrix

# Part I : Equal risk contribution

In [ ]:
# Question 1 - What is the objective function?
def obj_logw_long_only(w):
    obj = -np.sum(np.log(w))
    return obj

In [ ]:
# Question 2 - What is the associated gradient?
def grad_logw_long_only(w):
    grad = -1.0/w
    return grad

Constraint: portfolio risk (volatility) equals or is below target.
We set it as inequality: riskTarget - sqrt(w^T Σ w) >= 0.

In [8]:
def constr_volatility(w, riskTarget, covMatrix):
    # Question 3 - What is the volatility of the portfolio?
    risk = np.sqrt(np.dot(w, np.dot(covMatrix, w)))
    
    # Constraint function: riskTarget - risk must be >= 0.
    con_value = riskTarget - risk
    
    # Question 4 - What is the associated gradient?
    jac = -np.dot(covMatrix, w) / risk
    
    return con_value, jac

def constr_fun(w, riskTarget, covMatrix):
    con_val, _ = constr_volatility(w, riskTarget, covMatrix)
    return con_val

def constr_jac(w, riskTarget, covMatrix):
    jac = constr_volatility(w, riskTarget, covMatrix)
    return jac

In [16]:
# Optimisation code
numAssets = sigma.shape[1]
initial = np.ones(numAssets)
initial = initial / np.sum(initial)
riskTarget = np.sqrt(np.dot(initial, np.dot(sigma, initial)))

# Define bounds: each weight between 1e-6 and 1
bounds = [(1e-6, 1) for _ in range(numAssets)]
# Notes:
# Lower bound is used at 1e-6 and not 0 to ensure that weights remain > 0 for log function.


# Run optimization (long-only)
cons = {'type': 'ineq',
        'fun': lambda w: constr_fun(w, riskTarget, sigma),
        'jac': lambda w: np.asarray(constr_jac(w, riskTarget, sigma)[1]).ravel()}

res0 = minimize(fun = obj_logw_long_only,
                x0 = initial,
                method = 'SLSQP',
                jac = lambda w: grad_logw_long_only(w),
                bounds = bounds,
                constraints = cons,
                options = {'maxiter': 500, 'disp': False})

C:\Users\Emile\AppData\Local\Temp\ipykernel_8204\768812558.py:3: RuntimeWarning: invalid value encountered in log
  obj = -np.log(w @ mu) + 0.5 * w @ sigma @ w


In [18]:
print("Part I : Equal risk contribution")
# Question 5 & 6 - What is the optimized portfolio? What are the risk contributions of the assets? Check that the risk contributions of the assets are identical

# Normalize solution to sum to one
w_sol = res0.x / np.sum(res0.x)
print("Optimal weights:", w_sol)
# Compute risk contributions of each asset:
risk_contrib = w_sol * (np.dot(sigma, w_sol) / np.sqrt(np.dot(w_sol, np.dot(sigma, w_sol))))
print("Risk contributions:", risk_contrib)
# Compute total risk of portfolio:
total_risk = np.sqrt(np.dot(w_sol, np.dot(sigma, w_sol)))
print("Portfolio risk:", total_risk)

Part I : Equal risk contribution
Optimal weights: [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
Risk contributions: [0.0252804  0.0315992  0.03913088 0.04346526 0.05548188 0.07254213]
Portfolio risk: 0.2674997403781337


# Part II : Long / short optimisation

In [22]:
# Objective: minimize - sum( |mu| * log(|w|) )
def obj_logw_long_short(w, mu):
    obj = -np.sum(np.abs(mu) * np.log(np.abs(w)))
    return obj

def grad_logw_long_short(w, mu):
    grad = -np.abs(mu) / w
    return grad

# Use same volatility constraint as before (using riskTarget computed from initial weights)
# Set initial weights: +1 for mu>0, -1 for mu<0, normalized by count of positives
initial_ls = np.zeros(numAssets)
initial_ls[mu > 0] = 1
initial_ls[mu < 0] = -1
count_positive = np.sum(mu > 0)
initial_ls = initial_ls / count_positive
riskTarget_ls = np.sqrt(np.dot(initial_ls, np.dot(sigma, initial_ls)))

In [23]:
#Question 7 & 8  - Determine lb and ub
# Set bounds: for assets with mu > 0: [1e-6, 1], for mu < 0: [-1, -1e-6]
bounds_ls = [(1e-6, 1) if mu[i] > 0 else (-1, -1e-6) for i in range(numAssets)]

In [26]:
# Constraint remains the same form (volatility constraint)
cons_ls = {'type': 'ineq',
           'fun': lambda w: constr_fun(w, riskTarget_ls, sigma),
           'jac': lambda w: np.asarray(constr_jac(w, riskTarget, sigma)[1]).ravel()}

res1 = minimize(fun = obj_logw_long_short,
                x0 = initial_ls,
                args = (mu),
                method = 'SLSQP',
                jac = grad_logw_long_short,
                bounds = bounds_ls,
                constraints = cons_ls,
                options = {'maxiter': 500, 'disp': False})

print("Part II : Long / short optimisation")
w_ls = res1.x
print("Optimal long/short weights:", w_ls)
print("Sum weights:", np.sum(w_ls))

# (optionnel) risque et RC pour vérifier
total_risk_ls = np.sqrt(np.dot(w_ls, np.dot(sigma, w_ls)))
print("Portfolio risk:", total_risk_ls)

Part II : Long / short optimisation
Optimal long/short weights: [ 0.30210994 -0.24790741  0.43761774 -0.51670409  0.39819985 -0.2601219 ]
Sum weights: 0.11319413537237372
Portfolio risk: 0.25184235455759524


# Part III : Long / short optimisation with market neutrality constraint

In [27]:
# Define equality constraint: sum(w) == 0
def eq_market_neutral(w):
    return np.sum(w)

def eq_market_neutral_jac(w):
    return np.ones_like(w)

In [29]:
cons_eq = [{'type': 'ineq',
            'fun': lambda w: constr_fun(w, riskTarget_ls, sigma),
            'jac': lambda w: np.asarray(constr_jac(w, riskTarget, sigma)[1]).ravel()},
           {'type': 'eq',
            'fun': lambda w: eq_market_neutral(w),
            'jac': lambda w: np.asarray(constr_jac(w, riskTarget, sigma)[1]).ravel()}]

res2 = minimize(fun = obj_logw_long_short,
                x0 = initial_ls,
                args = (mu),
                method = 'SLSQP',
                jac = grad_logw_long_short,
                bounds = bounds_ls,
                constraints = cons_eq,
                options = {'maxiter': 500, 'disp': False})

In [30]:
print("Part III : Long / short optimisation with market neutrality constraint")
w_ls = res1.x          # Part II
w_lsmn = res2.x        # Part III

print("Weights (Part II - Long/Short):", w_ls)
print("Sum weights (Part II):", np.sum(w_ls))

print("Weights (Part III - L/S Market Neutral):", w_lsmn)
print("Sum weights (Part III, should be 0):", np.sum(w_lsmn))

# Compare risks
risk_ls = np.sqrt(w_ls @ sigma @ w_ls)
risk_lsmn = np.sqrt(w_lsmn @ sigma @ w_lsmn)
print("Risk (Part II):", risk_ls)
print("Risk (Part III):", risk_lsmn)

# Optional: compare gross exposure (|w| sum)
print("Gross exposure (Part II):", np.sum(np.abs(w_ls)))
print("Gross exposure (Part III):", np.sum(np.abs(w_lsmn)))

Part III : Long / short optimisation with market neutrality constraint
Weights (Part II - Long/Short): [ 0.30210994 -0.24790741  0.43761774 -0.51670409  0.39819985 -0.2601219 ]
Sum weights (Part II): 0.11319413537237372
Weights (Part III - L/S Market Neutral): [ 0.33333333 -0.33333333  0.33333333 -0.33333333  0.33333333 -0.33333333]
Sum weights (Part III, should be 0): 1.581845765485923e-11
Risk (Part II): 0.2518423545575953
Risk (Part III): 0.2518421022078009
Gross exposure (Part II): 2.1626609225885294
Gross exposure (Part III): 2.00000000001404
